In [26]:
import json
import pandas as pd
import numpy as np
import networkx as nx
import os
import shutil
import subprocess
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn import preprocessing
from sklearn.cluster import AgglomerativeClustering

In [43]:
len_data_arr = []
results_data_arr_agglo = []
results_data_arr_bunch = []
for root, dirs, files in os.walk('Results_21102020'):
    for file in files:
        #print(file)
        #print(file.split('_')[-1])
        if file.split('_')[-1] == 'len.txt':
            #print(file, 'len')
            proj_len_file = open('Results_21102020/' + file)
            proj_name = file.split('_')[0]
            proj_len = proj_len_file.readline().replace('\n','')
            len_data_arr.append([proj_name, proj_len])
            proj_len_file.close()
        elif file.split('_')[-1] == 'results.txt':
            #print(file, 'results')
            proj_result_file = open('Results_21102020/' + file)
            proj_name = file.split('_')[0]
            
            initial_line = 2 - 1
            initial_line_result = 5 - 1
            tmp_arr = []
            
            for i, line in enumerate(proj_result_file):
                
                if i == initial_line:
                    line = line.replace('MoJo', '')
                    line = line.replace('.rsf', '')
                    line = line.replace('(', '')
                    line = line.replace(')', '')
                    line = line.split('_')
                    name = line[1].split('/')[-1]
                    #version = line[0].split('-')[-1]
                    version = line[2]
                    #print(line)
                    ## If gathers results for bunch
                    if line[3] == 'exhaustive' or line[3] == 'hillclimbing' or line[3] == 'ga':
                        bunch_bool = True
                        method = line[3]
                        tmp_arr.append(name)
                        tmp_arr.append(version)
                        tmp_arr.append(method)
                        
                        initial_line += 5
                        
                    
                    
                    ## Else gathers results for agglomerative
                    else:
                        agglo_bool = True
                        n_cluster = line[3]
                        affinity = line[4]
                        linkage = line[5]
                        #print(line)
                        #print('Version', version)
                        initial_line += 5
                        tmp_arr.append(name)
                        tmp_arr.append(version)
                        tmp_arr.append(n_cluster)
                        tmp_arr.append(affinity)
                        tmp_arr.append(linkage)
                        
                elif i == initial_line_result:
                    n_MoJo = line.split(' ')[-1].split('\n')[0]

                    #print(line)
                    initial_line_result += 5
                    tmp_arr.append(n_MoJo)
                    #tmp_arr.append((1 - (float(n_MoJo) / float(num_class))) * 100)
                    
                    if agglo_bool == True:
                        results_data_arr_agglo.append(tmp_arr)
                        agglo_bool = False
                    elif bunch_bool == True:
                        results_data_arr_bunch.append(tmp_arr)
                        bunch_bool = False
                    tmp_arr = []
                    
                    
                
                

In [44]:
pd.DataFrame(results_data_arr_agglo)

,0,1,2,3,4,5
0,aws-aws-sdk-java-v2,2.9.9,5,euclidean,complete,1280
1,aws-aws-sdk-java-v2,2.9.9,5,euclidean,average,1695
2,aws-aws-sdk-java-v2,2.9.9,5,euclidean,single,644
3,aws-aws-sdk-java-v2,2.9.9,5,l1,complete,1070
4,aws-aws-sdk-java-v2,2.9.9,5,l1,average,695
...,...,...,...,...,...,...
62728,web3j-web3j,v4.5.2,25,manhattan,average,66
62729,web3j-web3j,v4.5.2,25,manhattan,single,59
62730,web3j-web3j,v4.5.2,25,cosine,complete,205
62731,web3j-web3j,v4.5.2,25,cosine,average,251


In [45]:
pd.DataFrame(results_data_arr_bunch)

,0,1,2,3
0,aws-aws-sdk-java-v2,2.9.9,hillclimbing,1421
1,aws-aws-sdk-java-v2,2.9.9,ga,1646
2,aws-aws-sdk-java-v2,2.9.9,exhaustive,1674
3,aws-aws-sdk-java-v2,2.9.8,hillclimbing,1406
4,aws-aws-sdk-java-v2,2.9.8,ga,1722
...,...,...,...,...
2336,web3j-web3j,v4.5.4,ga,378
2337,web3j-web3j,v4.5.4,exhaustive,373
2338,web3j-web3j,v4.5.2,hillclimbing,350
2339,web3j-web3j,v4.5.2,ga,362


In [46]:
pd.DataFrame(len_data_arr)

,0,1
0,aws-aws-sdk-java-v2,2168
1,bdew-neiaddons,38
2,bkromhout-realm-java,352
3,btraceio-btrace,495
4,bytedeco-javacpp,154
...,...,...
70,swagger-api-swagger-core,763
71,testcontainers-testcontainers-java,151
72,uber-NullAway,106
73,voxeolabs-moho,639


In [47]:
agglo_results = pd.DataFrame(results_data_arr_agglo)
agglo_results.columns = ['project_name', 'version', 'cluster_division', 'affinity', 'linkage', 'MoJo_results']
agglo_results.head()

,project_name,version,cluster_division,affinity,linkage,MoJo_results
0,aws-aws-sdk-java-v2,2.9.9,5,euclidean,complete,1280
1,aws-aws-sdk-java-v2,2.9.9,5,euclidean,average,1695
2,aws-aws-sdk-java-v2,2.9.9,5,euclidean,single,644
3,aws-aws-sdk-java-v2,2.9.9,5,l1,complete,1070
4,aws-aws-sdk-java-v2,2.9.9,5,l1,average,695


In [48]:
bunch_results = pd.DataFrame(results_data_arr_bunch)
bunch_results.columns = ['project_name', 'version', 'method', 'MoJo_results']
bunch_results.head()

,project_name,version,method,MoJo_results
0,aws-aws-sdk-java-v2,2.9.9,hillclimbing,1421
1,aws-aws-sdk-java-v2,2.9.9,ga,1646
2,aws-aws-sdk-java-v2,2.9.9,exhaustive,1674
3,aws-aws-sdk-java-v2,2.9.8,hillclimbing,1406
4,aws-aws-sdk-java-v2,2.9.8,ga,1722


In [49]:
proj_len_df = pd.DataFrame(len_data_arr)
proj_len_df.columns = ['project_name', 'len']
proj_len_df.head()

,project_name,len
0,aws-aws-sdk-java-v2,2168
1,bdew-neiaddons,38
2,bkromhout-realm-java,352
3,btraceio-btrace,495
4,bytedeco-javacpp,154


In [50]:
## Remove flyway error
agglo_results = agglo_results[agglo_results['project_name'] != 'flyway']
agglo_results = agglo_results[agglo_results['project_name'] != 'mrniko-netty-socketio']



In [51]:
#agglo_results.to_csv('agglo_results_21102020.csv', index=False)
#bunch_results.to_csv('bunch_results_21102020.csv', index=False)

In [52]:
proj_len_df.to_csv('proj_len.csv', index=False)

In [53]:
main_df_agglo = agglo_results.merge(proj_len_df, on='project_name', how='left')
main_df_agglo['len'] = main_df_agglo['len'].astype('int32')
main_df_agglo['cluster_division'] = main_df_agglo['cluster_division'].astype('int32')
main_df_agglo['MoJo_results'] = main_df_agglo['MoJo_results'].astype('int32')
main_df_agglo['num_clusters'] = main_df_agglo['len'] // main_df_agglo['cluster_division']
main_df_agglo['accuracy'] = 1-  (main_df_agglo['MoJo_results'] / main_df_agglo['len'])
main_df_agglo.head()

,project_name,version,cluster_division,affinity,linkage,MoJo_results,len,num_clusters,accuracy
0,aws-aws-sdk-java-v2,2.9.9,5,euclidean,complete,1280,2168,433,0.409594
1,aws-aws-sdk-java-v2,2.9.9,5,euclidean,average,1695,2168,433,0.218173
2,aws-aws-sdk-java-v2,2.9.9,5,euclidean,single,644,2168,433,0.702952
3,aws-aws-sdk-java-v2,2.9.9,5,l1,complete,1070,2168,433,0.506458
4,aws-aws-sdk-java-v2,2.9.9,5,l1,average,695,2168,433,0.679428


In [54]:
main_df_bunch = bunch_results.merge(proj_len_df, on='project_name', how='left')
main_df_bunch['len'] = main_df_bunch['len'].astype('int32')
main_df_bunch['MoJo_results'] = main_df_bunch['MoJo_results'].astype('int32')
main_df_bunch['accuracy'] = 1-  (main_df_bunch['MoJo_results'] / main_df_bunch['len'])
main_df_bunch.head()

,project_name,version,method,MoJo_results,len,accuracy
0,aws-aws-sdk-java-v2,2.9.9,hillclimbing,1421,2168,0.344557
1,aws-aws-sdk-java-v2,2.9.9,ga,1646,2168,0.240775
2,aws-aws-sdk-java-v2,2.9.9,exhaustive,1674,2168,0.227860
3,aws-aws-sdk-java-v2,2.9.8,hillclimbing,1406,2168,0.351476
4,aws-aws-sdk-java-v2,2.9.8,ga,1722,2168,0.205720


In [55]:
main_df_agglo = main_df_agglo[main_df_agglo['accuracy'] == main_df_agglo.groupby(['project_name', 'version'])['accuracy'].transform('max')]
main_df_agglo = main_df_agglo[main_df_agglo['affinity'] != 'l2']
main_df_agglo = main_df_agglo[main_df_agglo['affinity'] != 'l1']
main_df_agglo = main_df_agglo[main_df_agglo['accuracy'] != 1]
main_df_agglo.head()

,project_name,version,cluster_division,affinity,linkage,MoJo_results,len,num_clusters,accuracy
86,aws-aws-sdk-java-v2,2.9.9,25,manhattan,single,281,2168,86,0.870387
176,aws-aws-sdk-java-v2,2.9.8,25,manhattan,single,282,2168,86,0.869926
266,aws-aws-sdk-java-v2,2.9.7,25,manhattan,single,282,2168,86,0.869926
356,aws-aws-sdk-java-v2,2.9.6,25,manhattan,single,282,2168,86,0.869926
446,aws-aws-sdk-java-v2,2.9.5,25,manhattan,single,282,2168,86,0.869926


In [56]:
main_df_bunch = main_df_bunch[main_df_bunch['accuracy'] == main_df_bunch.groupby(['project_name', 'version'])['accuracy'].transform('max')]

In [83]:
### Visualization
visual_agglo_df = main_df_agglo.copy()
visual_agglo_df.columns = ['project_name', 'version', 'cluster_division', 'affinity', 'linkage', 'MoJo_results', 'num_classes', 'num_clusters', 'MoJoFM']
visual_agglo_df.drop(columns=['MoJo_results'], inplace=True)
visual_agglo_df['MoJoFM'] = visual_agglo_df['MoJoFM'] * 100

visual_agglo_df.sort_values(['MoJoFM'], ascending=False).head(10)

,project_name,version,cluster_division,affinity,linkage,num_classes,num_clusters,MoJoFM
11762,facebook-facebook-java-business-sdk,v3.2.7,25,euclidean,single,596,23,98.657718
45221,redisson-redisson,redisson-parent-3.11.6,25,manhattan,single,1401,56,98.144183
38894,oracle-oci-java-sdk,v1.7.0,25,cosine,single,4705,188,97.662062
38612,oracle-oci-java-sdk,v1.8.2,25,euclidean,single,4705,188,97.321998
38702,oracle-oci-java-sdk,v1.8.1,25,euclidean,single,4705,188,97.321998
38792,oracle-oci-java-sdk,v1.8.0,25,euclidean,single,4705,188,97.258236
38522,oracle-oci-java-sdk,v1.9.0,25,euclidean,single,4705,188,97.258236
38432,oracle-oci-java-sdk,v1.9.1,25,euclidean,single,4705,188,97.236982
11672,facebook-facebook-java-business-sdk,v3.2.8,25,euclidean,single,596,23,96.979866
11582,facebook-facebook-java-business-sdk,v3.2.9,25,euclidean,single,596,23,96.812081


In [84]:
### Visualization
visual_bunch_df = main_df_bunch.copy()
visual_bunch_df.columns = ['project_name', 'version', 'method', 'MoJo_results', 'num_classes', 'MoJoFM']
visual_bunch_df.drop(columns=['MoJo_results'], inplace=True)
visual_bunch_df['MoJoFM'] = visual_bunch_df['MoJoFM'] * 100

visual_bunch_df.sort_values(['MoJoFM'], ascending=False).head(10)

,project_name,version,method,num_classes,MoJoFM
661,google-openrtb,1.5.9,ga,44,88.636364
666,google-openrtb,1.5.7,hillclimbing,44,86.363636
948,javafunk-funk,funk-0.1.4,hillclimbing,268,85.820896
951,javafunk-funk,funk-0.1.3,hillclimbing,268,83.208955
676,google-openrtb,1.5.4,ga,44,81.818182
686,google-openrtb,1.5.12,exhaustive,44,81.818182
672,google-openrtb,1.5.5,hillclimbing,44,81.818182
563,google-dagger,dagger-2.3,exhaustive,1052,80.988593
498,facebook-react-native-fbsdk,0.7.0,hillclimbing,21,80.952381
496,facebook-react-native-fbsdk,0.8.0,ga,21,80.952381


In [64]:
agglo_results = main_df_agglo[['project_name', 'version', 'accuracy']]
agglo_results.columns = ['project_name', 'version', 'agglo_accuracy']
bunch_results = main_df_bunch[['project_name', 'version', 'accuracy']]
bunch_results.columns = ['project_name', 'version', 'bunch_accuracy']
combine_df = agglo_results.merge(bunch_results, on=['project_name', 'version'])

In [76]:
combine_df.sort_values(['agglo_accuracy'], ascending=False)

,project_name,version,agglo_accuracy,bunch_accuracy,algo_choice
789,facebook-facebook-java-business-sdk,v3.2.7,0.986577,0.500000,agglo
2924,redisson-redisson,redisson-parent-3.11.6,0.981442,0.194861,agglo
2534,oracle-oci-java-sdk,v1.7.0,0.976621,0.265037,agglo
2532,oracle-oci-java-sdk,v1.8.1,0.973220,0.268013,agglo
2531,oracle-oci-java-sdk,v1.8.2,0.973220,0.228905,agglo
...,...,...,...,...,...
3419,square-okhttp,parent-4.7.2,0.445122,0.408537,agglo
3418,square-okhttp,parent-4.8.0,0.445122,0.390244,agglo
3417,square-okhttp,parent-4.8.0,0.445122,0.390244,agglo
3416,square-okhttp,parent-4.8.0,0.445122,0.390244,agglo


In [77]:
combine_df.sort_values(['bunch_accuracy'], ascending=False)

,project_name,version,agglo_accuracy,bunch_accuracy,algo_choice
1493,google-openrtb,1.5.9,0.818182,0.886364,bunch
1497,google-openrtb,1.5.9,0.818182,0.886364,bunch
1492,google-openrtb,1.5.9,0.818182,0.886364,bunch
1499,google-openrtb,1.5.9,0.818182,0.886364,bunch
1498,google-openrtb,1.5.9,0.818182,0.886364,bunch
...,...,...,...,...,...
2883,pxb1988-dex2jar,0.0.9.14,0.945652,0.097826,agglo
2882,pxb1988-dex2jar,0.0.9.14,0.945652,0.097826,agglo
2881,pxb1988-dex2jar,0.0.9.14,0.945652,0.097826,agglo
2880,pxb1988-dex2jar,0.0.9.14,0.945652,0.097826,agglo


In [67]:
conditions = [
    (combine_df['agglo_accuracy'] > combine_df['bunch_accuracy']),
    (combine_df['bunch_accuracy'] >= combine_df['agglo_accuracy'])
]

values = ['agglo', 'bunch']

combine_df['algo_choice'] = np.select(conditions, values)

In [75]:
combine_df.sort_values(['agglo_accuracy'], ascending=False).head()

,project_name,version,agglo_accuracy,bunch_accuracy,algo_choice
789,facebook-facebook-java-business-sdk,v3.2.7,0.986577,0.500000,agglo
2924,redisson-redisson,redisson-parent-3.11.6,0.981442,0.194861,agglo
2534,oracle-oci-java-sdk,v1.7.0,0.976621,0.265037,agglo
2532,oracle-oci-java-sdk,v1.8.1,0.973220,0.268013,agglo
2531,oracle-oci-java-sdk,v1.8.2,0.973220,0.228905,agglo


In [70]:
combine_df.to_csv('combine_algo_choice_23102020.csv', index=False)

In [69]:
combine_df.groupby('algo_choice').count()

,project_name,version,agglo_accuracy,bunch_accuracy
algo_choice,,,,
agglo,3627,3627,3627,3627
bunch,47,47,47,47


In [74]:
print(len(combine_df['project_name'].unique()))
for element in combine_df['project_name'].unique():
    if element != 
    print(element)

71
aws-aws-sdk-java-v2
bdew-neiaddons
bkromhout-realm-java
btraceio-btrace
bytedeco-javacpp
codecentric-spring-boot-admin
codenvy-legacy-che-plugins
coobird-thumbnailator
cryptomator-cryptomator
dropwizard-dropwizard
dropwizard-metrics
evant-gradle-retrolambda
facebook-facebook-android-sdk
facebook-facebook-java-business-sdk
facebook-fresco
facebook-litho
facebook-react-native-fbsdk
google-cdep
google-dagger
google-error-prone
google-gitiles
google-openrtb-doubleclick
google-openrtb
grpc-grpc-java
havarunner-havarunner
hazelcast-hazelcast
immutables-immutables
ionic-team-capacitor
iSoron-uhabits
jankotek-mapdb
javafunk-funk
javaparser-javaparser
jboss-switchyard-release
jenkinsci-jenkins
jhy-jsoup
joelittlejohn-jsonschema2pojo
kijiproject-kiji-bento
knowm-XChange
lettuce-io-lettuce-core
mockito-mockito
mybatis-mybatis-3
naver-pinpoint
netty-netty
oblac-jodd
openengsb-openengsb
oracle-oci-java-sdk
oracle-opengrok
oracle-spacewalk-java
permissions-dispatcher-PermissionsDispatcher
pxb1988

In [16]:
ck_max = pd.read_csv('ck_max_194.csv')
ck_std = pd.read_csv('ck_std_194.csv')
ck_mean = pd.read_csv('ck_mean_194.csv')
ck_sum = pd.read_csv('ck_sum_194.csv')

In [17]:
ck_max_column = []
ck_std_column = []
ck_mean_column = []
ck_sum_column = []


for element in ck_max.columns:
    #print(element)
    if element != 'project_name':
        ck_max_column.append(element + '_max')
    else:
        ck_max_column.append('project_name')
        
        
for element in ck_std.columns:
    #print(element)
    if element != 'project_name':
        ck_std_column.append(element + '_std')
    else:
        ck_std_column.append('project_name')
        
for element in ck_mean.columns:
    #print(element)
    if element != 'project_name':
        ck_mean_column.append(element + '_mean')
    else:
        ck_mean_column.append('project_name')
        
for element in ck_sum.columns:
    #print(element)
    if element != 'project_name':
        ck_sum_column.append(element + '_sum')
    else:
        ck_sum_column.append('project_name')

        
ck_max.columns = ck_max_column
ck_std.columns = ck_std_column
ck_sum.columns = ck_sum_column
ck_mean.columns = ck_mean_column


ck_max.head()

,project_name,cbo_max,wmc_max,dit_max,rfc_max,lcom_max,totalMethods_max,staticMethods_max,publicMethods_max,privateMethods_max,...,numbersQty_max,assignmentsQty_max,mathOperationsQty_max,variablesQty_max,maxNestedBlocks_max,anonymousClassesQty_max,subClassesQty_max,lambdasQty_max,uniqueWordsQty_max,modifiers_max
0,apache-metamodel,57.0,160.0,6.0,162.0,2701.0,77.0,45.0,76.0,12.0,...,231.0,217.0,30.0,202.0,9.0,27.0,1.0,9.0,344.0,1025.0
1,alibaba-jetcache,41.0,77.0,6.0,110.0,2212.0,69.0,10.0,68.0,15.0,...,92.0,111.0,36.0,93.0,5.0,2.0,25.0,36.0,227.0,1025.0
2,evant-gradle-retrolambda,10.0,14.0,2.0,23.0,45.0,10.0,10.0,9.0,1.0,...,3.0,47.0,30.0,43.0,2.0,1.0,0.0,2.0,95.0,1.0
3,apache-qpid-broker-j,166.0,564.0,26.0,439.0,16972.0,193.0,35.0,113.0,79.0,...,788.0,329.0,108.0,315.0,10.0,42.0,24.0,22.0,4231.0,1025.0
4,apache-cxf,131.0,1168.0,27.0,424.0,26796.0,242.0,101.0,229.0,77.0,...,840.0,1278.0,253.0,604.0,9.0,19.0,38.0,32.0,619.0,1025.0


In [18]:
main_df_agglo = main_df_agglo.merge(ck_max, how='left', on='project_name')
main_df_agglo = main_df_agglo.merge(ck_mean, how='left', on='project_name')
main_df_agglo = main_df_agglo.merge(ck_std, how='left', on='project_name')
main_df_agglo = main_df_agglo.merge(ck_sum, how='left', on='project_name')

main_df_agglo.head()

,project_name,version,cluster_division,affinity,linkage,MoJo_results,len,num_clusters,accuracy,cbo_max,...,numbersQty_sum,assignmentsQty_sum,mathOperationsQty_sum,variablesQty_sum,maxNestedBlocks_sum,anonymousClassesQty_sum,subClassesQty_sum,lambdasQty_sum,uniqueWordsQty_sum,modifiers_sum
0,aws-aws-sdk-java-v2,2.9.9,25,manhattan,single,281,2168,86,0.870387,138.0,...,5944.0,16957.0,2103.0,15768.0,1194.0,243.0,823.0,3656.0,139293.0,117054.0
1,aws-aws-sdk-java-v2,2.9.8,25,manhattan,single,282,2168,86,0.869926,138.0,...,5944.0,16957.0,2103.0,15768.0,1194.0,243.0,823.0,3656.0,139293.0,117054.0
2,aws-aws-sdk-java-v2,2.9.7,25,manhattan,single,282,2168,86,0.869926,138.0,...,5944.0,16957.0,2103.0,15768.0,1194.0,243.0,823.0,3656.0,139293.0,117054.0
3,aws-aws-sdk-java-v2,2.9.6,25,manhattan,single,282,2168,86,0.869926,138.0,...,5944.0,16957.0,2103.0,15768.0,1194.0,243.0,823.0,3656.0,139293.0,117054.0
4,aws-aws-sdk-java-v2,2.9.5,25,manhattan,single,282,2168,86,0.869926,138.0,...,5944.0,16957.0,2103.0,15768.0,1194.0,243.0,823.0,3656.0,139293.0,117054.0


In [19]:
main_df_bunch = main_df_bunch.merge(ck_max, how='left', on='project_name')
main_df_bunch = main_df_bunch.merge(ck_mean, how='left', on='project_name')
main_df_bunch = main_df_bunch.merge(ck_std, how='left', on='project_name')
main_df_bunch = main_df_bunch.merge(ck_sum, how='left', on='project_name')

main_df_bunch.head()

,project_name,version,method,MoJo_results,len,accuracy,cbo_max,wmc_max,dit_max,rfc_max,...,numbersQty_sum,assignmentsQty_sum,mathOperationsQty_sum,variablesQty_sum,maxNestedBlocks_sum,anonymousClassesQty_sum,subClassesQty_sum,lambdasQty_sum,uniqueWordsQty_sum,modifiers_sum
0,aws-aws-sdk-java-v2,2.9.9,hillclimbing,1421,2168,0.344557,138.0,130.0,15.0,238.0,...,5944.0,16957.0,2103.0,15768.0,1194.0,243.0,823.0,3656.0,139293.0,117054.0
1,aws-aws-sdk-java-v2,2.9.8,hillclimbing,1406,2168,0.351476,138.0,130.0,15.0,238.0,...,5944.0,16957.0,2103.0,15768.0,1194.0,243.0,823.0,3656.0,139293.0,117054.0
2,aws-aws-sdk-java-v2,2.9.7,exhaustive,1453,2168,0.329797,138.0,130.0,15.0,238.0,...,5944.0,16957.0,2103.0,15768.0,1194.0,243.0,823.0,3656.0,139293.0,117054.0
3,aws-aws-sdk-java-v2,2.9.6,exhaustive,1473,2168,0.320572,138.0,130.0,15.0,238.0,...,5944.0,16957.0,2103.0,15768.0,1194.0,243.0,823.0,3656.0,139293.0,117054.0
4,aws-aws-sdk-java-v2,2.9.5,exhaustive,1060,2168,0.511070,138.0,130.0,15.0,238.0,...,5944.0,16957.0,2103.0,15768.0,1194.0,243.0,823.0,3656.0,139293.0,117054.0


In [20]:
main_df_agglo.to_csv('agglo_results_21102020.csv', index=False)
main_df_bunch.to_csv('bunch_results_21102020.csv', index=False)

In [21]:
main_df_agglo = pd.read_csv('agglo_results_21102020.csv')
main_df_bunch = pd.read_csv('bunch_results_21102020.csv')

In [22]:
### Following is to change to Dr Aldeida's format
dr_main_df_agglo = main_df_agglo
dr_main_df_agglo['project'] = dr_main_df_agglo['project_name'] + dr_main_df_agglo['version']
dr_main_df_agglo['config'] = dr_main_df_agglo['cluster_division'].astype(str) + '_' + dr_main_df_agglo['linkage'] + '_' + dr_main_df_agglo['affinity']

dr_main_df_agglo.drop('project_name', axis=1, inplace=True)
dr_main_df_agglo.drop('version', axis=1, inplace=True)
dr_main_df_agglo.drop('cluster_division', axis=1, inplace=True)
dr_main_df_agglo.drop('affinity', axis=1, inplace=True)
dr_main_df_agglo.drop('linkage', axis=1, inplace=True)
dr_main_df_agglo.drop('MoJo_results', axis=1, inplace=True)
dr_main_df_agglo.drop('len', axis=1, inplace=True)
dr_main_df_agglo.drop('num_clusters', axis=1, inplace=True)
dr_main_df_agglo.drop('accuracy', axis=1, inplace=True)

dr_main_df_agglo.head()

print(dr_main_df_agglo.columns)


Index(['cbo_max', 'wmc_max', 'dit_max', 'rfc_max', 'lcom_max',
       'totalMethods_max', 'staticMethods_max', 'publicMethods_max',
       'privateMethods_max', 'protectedMethods_max',
       ...
       'mathOperationsQty_sum', 'variablesQty_sum', 'maxNestedBlocks_sum',
       'anonymousClassesQty_sum', 'subClassesQty_sum', 'lambdasQty_sum',
       'uniqueWordsQty_sum', 'modifiers_sum', 'project', 'config'],
      dtype='object', length=162)


In [23]:
### Following is to change to Dr Aldeida's format
dr_main_df_bunch = main_df_bunch
dr_main_df_bunch['project'] = dr_main_df_bunch['project_name'] + dr_main_df_bunch['version']
dr_main_df_bunch['config'] = dr_main_df_bunch['method']

dr_main_df_bunch.drop('project_name', axis=1, inplace=True)
dr_main_df_bunch.drop('version', axis=1, inplace=True)
dr_main_df_bunch.drop('method', axis=1, inplace=True)
dr_main_df_bunch.drop('MoJo_results', axis=1, inplace=True)
dr_main_df_bunch.drop('len', axis=1, inplace=True)
dr_main_df_bunch.drop('accuracy', axis=1, inplace=True)

dr_main_df_bunch.head()

print(dr_main_df_bunch.columns)


Index(['cbo_max', 'wmc_max', 'dit_max', 'rfc_max', 'lcom_max',
       'totalMethods_max', 'staticMethods_max', 'publicMethods_max',
       'privateMethods_max', 'protectedMethods_max',
       ...
       'mathOperationsQty_sum', 'variablesQty_sum', 'maxNestedBlocks_sum',
       'anonymousClassesQty_sum', 'subClassesQty_sum', 'lambdasQty_sum',
       'uniqueWordsQty_sum', 'modifiers_sum', 'project', 'config'],
      dtype='object', length=162)


In [25]:
dr_main_df_agglo.to_csv('draldeida_agglo_results_21102020.csv', index=False)
dr_main_df_bunch.to_csv('draldeida_bunch_results_21102020.csv', index=False)

In [28]:
main_df_agglo.head()

,project_name,version,cluster_division,affinity,linkage,MoJo_results,len,num_clusters,accuracy,cbo_max,...,numbersQty_sum,assignmentsQty_sum,mathOperationsQty_sum,variablesQty_sum,maxNestedBlocks_sum,anonymousClassesQty_sum,subClassesQty_sum,lambdasQty_sum,uniqueWordsQty_sum,modifiers_sum
0,aws-aws-sdk-java-v2,2.9.9,25,manhattan,single,281,2168,86,0.870387,138.0,...,5944.0,16957.0,2103.0,15768.0,1194.0,243.0,823.0,3656.0,139293.0,117054.0
1,aws-aws-sdk-java-v2,2.9.8,25,manhattan,single,282,2168,86,0.869926,138.0,...,5944.0,16957.0,2103.0,15768.0,1194.0,243.0,823.0,3656.0,139293.0,117054.0
2,aws-aws-sdk-java-v2,2.9.7,25,manhattan,single,282,2168,86,0.869926,138.0,...,5944.0,16957.0,2103.0,15768.0,1194.0,243.0,823.0,3656.0,139293.0,117054.0
3,aws-aws-sdk-java-v2,2.9.6,25,manhattan,single,282,2168,86,0.869926,138.0,...,5944.0,16957.0,2103.0,15768.0,1194.0,243.0,823.0,3656.0,139293.0,117054.0
4,aws-aws-sdk-java-v2,2.9.5,25,manhattan,single,282,2168,86,0.869926,138.0,...,5944.0,16957.0,2103.0,15768.0,1194.0,243.0,823.0,3656.0,139293.0,117054.0


In [90]:
g = main_df_agglo.groupby('project_name')

first = g['project_name', 'version'].head(1)
first.columns = ['project_name', 'firstRelease']

last = g['project_name', 'version'].tail(1)
last.columns = ['project_name', 'lastRelease']


project_first_last = first.merge(last, on='project_name')
#project_first_last = (pd.concat([g.head(1), g.tail(1)]).reset_index(drop=True).sort_values('project_name'))
pd.set_option('display.max_rows', 999)
pd.set_option('display.width', None)
project_first_last.head(71)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,project_name,firstRelease,lastRelease
0,aws-aws-sdk-java-v2,2.9.9,2.9.24
1,bdew-neiaddons,v1.9.4,v1.8.0
2,bkromhout-realm-java,v0.89.0,v0.87.1
3,btraceio-btrace,v1.3.9,v1.3.4
4,bytedeco-javacpp,1.5.3,1.4
5,codecentric-spring-boot-admin,1.5.7,1.4.5
6,codenvy-legacy-che-plugins,3.9.5,3.13.4.4
7,coobird-thumbnailator,0.4.9,0.4.10
8,cryptomator-cryptomator,1.5.6,1.5.0-beta3
9,dropwizard-dropwizard,v2.0.9,v2.0.11


In [48]:
len(project_first_last)

71

In [50]:
g_bunch = main_df_bunch.groupby('project_name')

first_bunch = g_bunch['project_name', 'version'].head(1)
first_bunch.columns = ['project_name', 'startRelease']

last_bunch = g_bunch['project_name', 'version'].tail(1)
last_bunch.columns = ['project_name', 'endRelease']


project_first_last_bunch = first_bunch.merge(last_bunch, on='project_name')
#project_first_last = (pd.concat([g.head(1), g.tail(1)]).reset_index(drop=True).sort_values('project_name'))
project_first_last_bunch.head(10)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,project_name,startRelease,endRelease
0,aws-aws-sdk-java-v2,2.9.9,2.9.24
1,bdew-neiaddons,v1.9.4,v1.7.1
2,bkromhout-realm-java,v0.89.0,v0.87.1
3,btraceio-btrace,v1.3.9,v1.3.4
4,bytedeco-javacpp,1.5.3,1.4
5,codecentric-spring-boot-admin,1.5.7,1.4.5
6,codenvy-legacy-che-plugins,3.9.5,3.13.4.4
7,coobird-thumbnailator,0.4.9,0.4.10
8,cryptomator-cryptomator,1.5.6,1.5.0-beta3
9,dropwizard-dropwizard,v2.0.9,v2.0.11


In [51]:
len(project_first_last_bunch)

74

In [52]:
def Diff(li1, li2):
    return (list(list(set(li1)-set(li2)) + list(set(li2)-set(li1))))
 
# Driver Code
#li1 = [10, 15, 20, 25, 30, 35, 40]
#li2 = [25, 40, 35]
#print(Diff(li1, li2))


print(Diff(project_first_last_bunch['project_name'], project_first_last['project_name']))

['hibernate-hibernate-orm', 'realm-realm-java', 'mrniko-netty-socketio']


In [53]:
project_first_last.to_csv('projectStartEndReleaseAgglo.csv', index=False)
project_first_last_bunch.to_csv('projectStartEndReleaseBunch.csv', index=False)